### Pycrash - Single Degree of Freedom Model
#### Demonstration using load cell barrier data
- see Demo - Basic Functions

In [1]:
# commands for Jupyter Notebook
# %% allow reloading of modules
%load_ext autoreload
%autoreload 2

#### Import modules necessary for analyses

In [2]:
import pandas as pd
pd.options.display.max_columns = None
import plotly.io as pio
pio.renderers.default = "browser"  # <- determines how plots are displayed using Plotly

#### Import Pycrash modules

In [8]:
import sys
sys.path.insert(0, '//Users//joe//Documents//pycrash//')

In [9]:
import pycrash
from pycrash.sdof_model import SDOF_Model
from pycrash.project import Project, project_info, load_project
from pycrash.vehicle import Vehicle
from pycrash.visualization.sdof_plot import plot_velocity, plot_acceleration, plot_fdx, plot_vehicle_fdx
from pycrash.functions.ar import cipriani_rest # get restitution

Current values for defined constants:
maximum available friction (mu_max) = 0.76
time step for vehicle motion (dt) = 0.01 s
No Environment File Provided
No Environment File Provided


## Load project data
- result will show saved data related to the project
- if error, the current working directory is not set appropriately.  Use `os.chcwd('\\path')` to set current
directory to the directory containing the project directory.

In [10]:
project_info('validation sdof')

This saved project contains:
Object of type "project" with name "validation sdof"
list objects in this order for loading project: ['validation sdof']
Example: project_name, veh1, veh2 = load_project('project_name')


#### Load Project Data

In [11]:
proj = load_project('validation sdof')

### Create Vehicle 1:
- "Vehicle" stores information about a single vehicle - all possible inputs do not need to be entered
- creating a Vehicle requires a "name" which is used to identify the vehicle in outputs / plots etc.

In [12]:
# minimal vehicle data is needed for SDOF model:

In [13]:
veh1 = Vehicle('Striking')
veh1.year = 1975
veh1.make = 'Foo'
veh1.model = 'Bar'
veh1.weight = 3500
veh1.brake = 0
veh1.vx_initial = 5

In [14]:
veh2 = Vehicle('Struck')
veh2.year = 2020
veh2.make = 'Honda'
veh2.model = 'Civic'
veh2.weight = 3330
veh2.brake = 0
veh2.vx_initial = 0

- The single degree of freedom module is designed to work in sets of three so that you can create an average, low, high at the same time.
- inputs are used in lists of 3

In [15]:
# create model inputs:
v1_vx_initial = [5, 7, 10]  # initial speeds for striking vehicle
run_list_names = ['Low', 'Average', 'High']
cor_list = [cipriani_rest(x) for x in v1_vx_initial]  # low restitution from sideswipe
# stiffness values determined in Demo = Basic Functions
k_mutual = 27734
k_mutual_low = k_mutual - 9512
k_mutual_high = k_mutual + 9512
k_model_list = [k_mutual_low, k_mutual, k_mutual_high]
models =[None] * len(run_list_names)  # create empty list for model runs

k_veh2 = 51479  # stiffness for rear of Vehicle 2
k_veh2_low = 26145
k_veh2_high = 97875
veh2_stiffnes_list = [k_veh2_low, k_veh2, k_veh2_high]

#### Run three models in loop:
- `tstop` - this determines the time to stop the model.  It's usually desired to have all models run to the same time.
- first, run all models with `tstop = None` then, once the maximum time is know, use that time [ms] as `tstop`

In [16]:
for i in range(len(v1_vx_initial)):

    model_inputs = {"name": run_list_names[i],
            "k": k_model_list[i],
            "cor": cor_list[i],
            "tstop": 0.117
        }
    # closing speed
    veh1.vx_initial = v1_vx_initial[i]
    # assign individual stiffness values to get vehicle-specific crush
    veh1.k = 60127
    veh2.k = veh2_stiffnes_list[i]
    models[i] = SDOF_Model(veh1, veh2, model_inputs=model_inputs)


------------ Model Inputs ---------------
Model Run = Low
Coefficient of Restitution = 0.3826279331330557
Constant Mutual Stiffness = 18222 lb/ft 
Model will run until t = 0.117 seconds

|------------ Vehicle Inputs -----------|

<- Vehicle 1 ->

Striking braking at 0%
Striking initial speed is 5 mph

<- Vehicle 2 ->

Struck braking at 0%
Struck initial speed is 0 mph
Model Closing Speed = 5 mph

|----------- Input Complete ------------|


Model Initiated ============>
===== Seperation ===========>
Time (s) = 0.08600000000000001
Peak Mutual Crush (in) = 4.82
Peak Force (lb) = 7312.36

========> Stopped t (s) = 0.118
Vehicle 1 crush determined using provided stiffness
Vehicle 2 crush determined using provided stiffness

------------ Model Inputs ---------------
Model Run = Average
Coefficient of Restitution = 0.33941271957714714
Constant Mutual Stiffness = 27734 lb/ft 
Model will run until t = 0.117 seconds

|------------ Vehicle Inputs -----------|

<- Vehicle 1 ->

Striking braking a

#### Plot force-displacement response for the three models

In [ ]:
plot_fdx(models)

In [ ]:
# assign individual stiffness values to get vehicle-specific crush

plot_vehicle_fdx(models)  # <- vehicle specific Fdx

In [ ]:
# plot velocity
plot_velocity(models, fill_diff = True, show_legend = False)

In [ ]:
# plot acceleration
plot_acceleration(models, fill_diff = True, show_legend = False)